In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Data Preprocessing

categorical_columns = ['sex', 'smoker', 'region']
for column in categorical_columns:
    print(f"\n{column} unique values:")
    print(dataset[column].unique())

dataset_encoded = pd.get_dummies(dataset, columns=['sex', 'region'], drop_first=True)
dataset_encoded['smoker'] = dataset_encoded['smoker'].map({'yes': 1, 'no': 0})

print("\nEncoded dataset sample:")
print(dataset_encoded.head())

In [ ]:
# Normalize the numerical features
numeric_features = ['age', 'bmi', 'children']
feature_mean = dataset_encoded[numeric_features].mean()
feature_std = dataset_encoded[numeric_features].std()
dataset_encoded[numeric_features] = (dataset_encoded[numeric_features] - feature_mean) / feature_std

In [ ]:
train_dataset = dataset_encoded.sample(frac=0.8, random_state=0)
test_dataset = dataset_encoded.drop(train_dataset.index)

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

print("Training set shape:", train_dataset.shape)
print("Testing set shape:", test_dataset.shape)

In [ ]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(train_dataset.columns)]),
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

model = build_model()
model.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
    train_dataset, train_labels,
    epochs=100,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stop, tfdocs.modeling.EpochDots()])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
print(hist.tail())

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.xlabel('Epoch')
plt.ylabel('Mean Abs Error [expenses]')
plt.plot(hist['epoch'], hist['mae'],
         label='Train Error')
plt.plot(hist['epoch'], hist['val_mae'],
         label='Val Error')
plt.legend()

plt.subplot(1, 2, 2)
plt.xlabel('Epoch')
plt.ylabel('Mean Square Error [expenses^2]')
plt.plot(hist['epoch'], hist['mse'],
         label='Train Error')
plt.plot(hist['epoch'], hist['val_mse'],
         label='Val Error')
plt.legend()
plt.show()

loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)
print('\nTesting set Mean Abs Error: {:5.2f} expenses'.format(mae))

if mae < 3500:
    print("Model passes the challenge requirement!")
else:
    print("Model needs improvement. The MAE should be less than 3500.")

test_predictions = model.predict(test_dataset).flatten()

plt.figure(figsize=(10, 10))
a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)
plt.title('Predictions vs Actual Values')
plt.show()

errors = test_predictions - test_labels
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=50)
plt.xlabel('Prediction Error [expenses]')
plt.ylabel('Count')
plt.title('Prediction Error Distribution')
plt.show()

print(f"Mean Absolute Error: ${mae:.2f}")
print(f"Mean Squared Error: ${mse:.2f}")
print(f"Root Mean Squared Error: ${np.sqrt(mse):.2f}")

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
